<div style='background-image: url("../share/Aerial_view_LLNL.jpg") ; padding: 0px ; background-size: cover ; border-radius: 15px ; height: 250px; background-position: 0% 80%'>
    <div style="float: center ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.8) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.9) ; line-height: 100%">Notebook 5:</div>
            <div style="font-size: x-large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)">Statistics for Digital Humanists</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)">Estimated Time: 60 minutes</div>
        </div>
    </div>
</div>

# Introduction to Statistics
In this jupyter notebook, we will learn some basic statistics, cover the basics of linear regression, and, finally, examine word counts in a text:

1. [Mean, Median, Mode](#1)
2. [Standard Deviation](#2)
3. [T-test](#3)
4. [Correlation](#4)
5. [Intro to Regression](#5)
6. [Frequenices of Words](#6)

## 0. Preparing Our Dataset  <a id='0'></a>
We will use some metadata on manuscripts.

For the most part, you don't have to worry about all the codes we used in this section to fix the data and make it more usable for out purpose of learning basic statistic. Instead, you can skip to the end of this section 0 and try to familiarize yourself with the [cleaned dataset](#clean).

In [ ]:
# This simply sets up the notebook. Don't worry about this for now.

import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
import seaborn as sns
from scipy.stats import ttest_ind
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('data/BSB-HS.csv')

In [ ]:
df.head()

In [ ]:
def get_year(x):
    
    if pd.isnull(x):
        return None
    
    pattern0 = r'([0-9]+)\./([0-9]+)\.\sJh\.'
    if re.search(pattern0, x) != None:
        tot = int(re.search(pattern0, x).groups()[0] + '00') + int(re.search(pattern0, x).groups()[1] + '00')
        return tot/2

    pattern3 = r'1\.\sHälfte\s([0-9]+)\.\sJh\.'
    if re.search(pattern3, x) != None:
        return int(re.search(pattern3, x).groups()[0] + '00') - 75
    
    pattern4 = r'2\.\sHälfte\s([0-9]+)\.\sJh\.'
    if re.search(pattern4, x) != None:
        return int(re.search(pattern4, x).groups()[0] + '00') - 25
    
    pattern5 = r'Mitte\s([0-9]+)\.\sJh\.'
    if re.search(pattern5, x) != None:
        return int(re.search(pattern5, x).groups()[0] + '00') - 50

    pattern6 = r'Ende\s([0-9]+)\.\sJh\.'
    if re.search(pattern6, x) != None:
        return int(re.search(pattern6, x).groups()[0] + '00') - 10
    
    pattern7 = r'1\.\sViertel\s([0-9]+)\.\sJh\.'
    if re.search(pattern7, x) != None:
        return int(re.search(pattern7, x).groups()[0] + '00') - 80
    
    pattern8 = r'2\.\sViertel\s([0-9]+)\.\sJh\.'
    if re.search(pattern8, x) != None:
        return int(re.search(pattern8, x).groups()[0] + '00') - 60
    
    pattern9 = r'3\.\sViertel\s([0-9]+)\.\sJh\.'
    if re.search(pattern9, x) != None:
        return int(re.search(pattern9, x).groups()[0] + '00') - 40
    
    pattern10 = r'4\.\sViertel\s([0-9]+)\.\sJh\.'
    if re.search(pattern10, x) != None:
        return int(re.search(pattern10, x).groups()[0] + '00') - 20
    
    pattern1 = r'([0-9]+)\.\sJh\.'
    if re.search(pattern1, x) != None:
        return int(re.search(pattern1, x).groups()[0] + '00') - 100
    
    pattern2 = r'([0-9]{4})'
    if re.search(pattern2, x) != None:
        return int(re.search(pattern2, x).groups()[0])
    
    else:
        return None
    
def get_extent(x):
    
    if pd.isnull(x):
        return None
    
    pattern = r'([0-9]+)\sBl\.'
    if re.search(pattern, x) != None:
        return int(re.search(pattern, x).groups()[0])
    
    else:
        return None
    
    
def fix_lang(x):
    if type(x) == str and len(x) > 0:
        return x
    else:
        return None

In [ ]:
df['Year'] = df['Creation'].apply(get_year)
df['Pages'] = df['Extent'].apply(get_extent)
df['Area'] = df['Height'] * df['Width']
df['Language'] = df['Language'].apply(fix_lang)

### Cleaned Dataset: <a id='clean'></a>

From the cleaned version of the dataset, we will isolate the columns we will use and create a new table called `manuscripts.` We will be working with this to learn basic statistics. Take a look at the first 5 rows of our dataset below:

In [ ]:
manuscripts = df[['Year', 'Pages', 'Height', 'Width', 'Area','Language']]
manuscripts.head()

## 1. Mean, Median, Mode  <a id='1'></a>

Let's review the definitions of the three main measures of central tendency: **Mean, Median, Mode**

### Mean

The mean of a set of values is the quantity given by the following formula:

$$
\frac{\text{Sum of values}}{\text{# of values}}
$$


Let's say we want to figure out the average number of pages from all the manuscripts in our dataset. To find this, we first isolate the `Pages` coumn and apply the `mean()` function:

In [ ]:
manuscripts['Pages'].mean()

### Median

The "middle" value of a dataset. That is, if you make an ordered list of the values from smallest to largest, the median is the middle value. If the number of values is even, then the median is the average of the 2 middle values. Similar to `mean`, there is a `median()` function that you can call on a set of values.

In [ ]:
manuscripts['Pages'].median()

### Mode

The mode of a set of values is a value that occurs the most frequently. There can be multiple modes in a dataset.

In [ ]:
manuscripts['Pages'].mode()

For a quick exercise, find the mean, median, and mode of the `Year` column of the manuscripts dataset.

In [ ]:
# Replace the ellipses with your own code
year_mean = ...
year_median = ...
year_mode = ...

print("The mean of the Year column is: ", year_mean)
print("The median of the Year column is: ", year_median)
print("The mode of the Year column is: ", year_mode)

Great! But that's too easy. And it really doesn't give much insight. What if we want to find out the average page of manuscripts in each of the different languages?

First we isolate the two columns we need: `Language` and `Pages`. Then we apply the `dropna()` function that removes the rows with missing values. Then, we use the `groupby()` function and put `Language` as a parameter, indicating we are "grouping" by the `Language` column. Finally, we select the `Pages` column and again use the `mean()` function to calculate the average *by Language*.

In [ ]:
manuscripts[['Language', 'Pages']].dropna().groupby(['Language'])['Pages'].mean()

Awesome! Now, can you try figuring out how to get the average `Area` by `Language`?

In [ ]:
# Your code here


Cool! Now that we know how to do some basic calculations, we can do more interesting things: data visualization! One of the most useful visualization for statistics is called a **boxplot**. Boxplots are a standardized way of displaying the distribution of data:
![a](https://pro.arcgis.com/en/pro-app/help/analysis/geoprocessing/charts/GUID-0E2C3730-C535-40CD-8152-80D794A996A7-web.png)


We can create a [boxplot](https://seaborn.pydata.org/generated/seaborn.boxplot.html) of the number of pages of each manuscript by langauge:

In [ ]:
sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize=(11.7, 8.27))
ax = sns.boxplot(ax = ax, data=manuscripts.dropna(),x='Pages', y='Language')

Take a moment to study the boxplot and lcoate the minimum, median, and maximum number of pages by language. You can also start to compare the average number of pages of manuscripts found in different languages by looking at where the medians are. What language has, on average, the most number of pages per manuscripts?

Let's look at another one. Here's a boxplot of the height of each manuscript by langauge:

In [ ]:
fig, ax = plt.subplots(figsize=(11.7, 8.27))
ax = sns.boxplot(ax = ax, data=manuscripts.dropna(),x='Height', y='Language')

Now it's your turn! Try creating a boxplot of the `Width` of the manuscripts by `Language`.

In [62]:
# Your code here


How about a box plot of the `Height` of the manuscripts by `Language`?

In [63]:
# Your code here


Awesome! Boxplots are powerful visualizations that can give a quick overview of that data's basic make up, making it also useful for comparison.

## 2. Standard Deviation (SD)  <a id='2'></a>

Standard deviation (SD) "measures roughly how far the numbers on the list are from their average" by calculating the root mean square of deviations from average:

$$
=\sqrt{\frac{\text{Sum[(value$_{1}$ - average)$^2$ + (value$_{2}$ - average)$^2$ ... (value$_{n}$ - average)$^2$]}}{n}}
$$

where $n$ is the number of values in the dataset.

To calculate the SD of a number we simply use the `np.std()`. For example, to find out the SD of the number of pages of all the greek manuscripts, we will use the following code:

In [ ]:
greek_manuscipts = manuscripts.loc[df['Language'] == 'Greek'].dropna()
np.std(greek_manuscipts["Pages"])

What we can infer from this is that the average Greek manuscript's page length is roughly 121 pages more or less than the mean of all page lengths of all Greek manuscripts, which is 161 pages. 

Below, we can see all the SDs for each of the columns.

In [ ]:
np.std(greek_manuscipts)

## 3. T-Test  <a id='3'></a>

[T-tests](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) are a method of testing hypotheses in statistics. They are used to help determine if there is a significant likelihood that the means of two independent populations differ.

To illustrate this concept, let's first come up with a research question. Suppose we wanted to know if there exists a significant difference in average page length between Latin and German manuscripts.

**Our null hypothesis**: There is no substantial difference in average page length between Latin and German manuscripts in general. That is, the difference between average page lengths is 0. 

Using our samples of Latin and German manuscripts, we can proceed to use a T-test to test this hypothesis.


Let's first select the page lengths for all German manuscripts in our data:

In [ ]:
group1 = manuscripts[manuscripts['Language']=='German'][['Language', 'Pages']].dropna()
group1.head()

and all Latin manuscripts:

In [ ]:
group2 = manuscripts[manuscripts['Language']=='Latin'][['Language', 'Pages']].dropna()
group2.head()

Now that we have two different, independent samples, we can use the function `ttest_ind`, which takes in 2 arrays, one for each group, to perform the T-test:

In [ ]:
ttest_ind(group1['Pages'], group2['Pages'])

Notice that the **p-value** is less than .005.

The p-value is essentially the probability that the result we observed happened by chance. Thus, the lower the p-value, the higher the likelihood that the mean between Latin and German manuscripts in general differs from 0. 

In addition, we say we "reject the null hypothesis" when the p-value is below a certain threshold. 

> If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages.

So, in this case, we can reject the null hypothesis at the 1% (p-value less than 0.01) threshold, and we can be confident that there is indeed a difference in average page length between Latin and German manuscripts in general.

## 4. Correlation <a id='4'></a>

Correlation measures the strength of a linear association between two variables. We want to know: what happens to one variable as the other increases? Is there an obvious linear pattern? 

* If one variable increases when the other increases, the two variables are **positively correlated**. 
* Conversely, when one variable increases when the other decreases, the two variables are **negatively correlated**.



Let's look at some examples.

### Perfect Positive Correlation

In [ ]:
# Generating dummy data
pos_x = np.arange(0, 50)
pos_y = 3*pos_x + 5
sns.regplot(x=pos_x, y=pos_y, color="blue")
plt.xlim(0, 50)

### Perfect Negative Correlation

In [ ]:
neg_x = np.arange(0, 50)
neg_y = -4*neg_x + 5
sns.regplot(x=neg_x, y=neg_y, color="red")
plt.xlim(0, 50)

As you can see above, perfect correlation implies that the relationship between the two variables can be defined by a line. However, perfect correlation rarely occurs in the real world. It's more likely that you will see data such as the following:

In [ ]:
ax = sns.regplot(x="Height", y="Width", data=manuscripts, color="green")

Although it isn't perfect correlation, the variables Height and Width do seem to be strongly positively correlated. 

In [ ]:
ax = sns.regplot(x="Year", y="Pages", data=manuscripts, color = "orange")

On the other hand, the Year and Pages variables show little to no correlation.

The correlation between two variables can be quantified by $r$, the **correlation coefficient**, where

$$
-1 \leq r \leq 1
$$

That is, $r$ ranges from -1 to 1, depending on the direction and strength of correlation. 

> **Direction**: The sign of $r$ denotes where the correlation is positive or negative. The coefficient is 1 when the two variables are perfectly positively correlated and -1 when the two variables are perfectly negatively correlated.

> **Strength**: The closer $r$ is to 0, the less the two variables are correlated. In other words, the greater the absolute value of $r$, the greater the correlation.

Now, we can ask, what is the correlation between Height and Width?

With one command, we can create a correlation matrix, showing the correlation between each variable in our table with every other variable. Notice that each variable is perfectly correlated with itself, as expected.

In [ ]:
manuscripts.corr()

As a quick exercise, take a look at the table and find a pair variables that:
* positively correlated
* negatively correlated
* strongly correlated (positive or negative)

## 5. Intro to Regression  <a id='5'></a>

This section will cover the basics of Linear Regression, a method in the social sciences that is used to determine the effect of an **independent variable** on a **dependent variable**. In other words, we want to find out whether a change in an independent variable changes the dependent variable.

### Example

Say we want to find out the effect of a manuscript's width on its area. In this case, the width is the independent variable and the area is the dependent variable. Then, we run a regression where the dependent variable is "regressed on" the independent variable. In this case, Area will be regressed on Width.

In [ ]:
import statsmodels.formula.api as smf

The library we will be using is called `statsmodels`. First, let's specify the regression formula/model. This is given in the form:

$$
\text{formula = dependent variable ~ independent variable}
$$

Since we are regressing Area on Width, our formula will be $\text{Area ~ Width} $, where $\text{Area}$ and $\text{Width}$ are columns in our `manuscripts` table from earlier.

In [ ]:
mod = smf.ols(formula='Area ~ Width', data=manuscripts)

Note that you enter the formula as a string. 

Now, let's "fit" our model to the data. What this code does is that, if we plotted Area and Width on a graph, it will fit a line through the points, similar to what we saw earlier in the correlation section. 

In [ ]:
reg = mod.fit()

Let's look at the regression summary, which will give us the information we were looking for: What is the effect of Width on the Area of a manuscript? 

In [ ]:
print(reg.summary())

There is a lot of information in the regression summary, but don't get overwhelmed. We will only be focusing on a couple details for our purposes today.

* Look at the **coef** column in the middle panel from the top. Look at the value in Width row. This is roughly the average change in area for a one-unit change in Width.

* Also in the Width row in the middle panel, look at the column labeled **P > |t|**. Recall the T-tests from earlier. This is the p-value of a T-test performed on the coefficient. What you need to know is that, if the p-value is below a certain threshold, say .05 or .01, then you can reject the null hypothesis that the Width of a manuscript is not associated with its Area.

Please ask the lab assistant(s) for help if you are having trouble with these concepts. They are not always easy to pick up the first time you are introduced to them. 

### Your Turn

Find two (numerical) columns, one dependent variable and one independent variable, from the `manuscripts` table and perform a regression. 

In [ ]:
# Replace the ellipses with your regression formula
mod = smf.ols(formula= ..., data=manuscripts)
res = mod.fit()
print(res.summary())

Now interpret the results based on what we discussed earlier. Edit the markdown cell immediately below.

*This is a Markdown cell. Discuss your findings here*

## 6. Frequenices of Words <a id='6'></a>

In this final section, we will look at how to calculate and plot word frequencies in a text.

Let's first read in our example text file `Anon_Gawain.txt`:

In [ ]:
with open('data/Anon_Gawain.txt', 'r') as f:
    text = f.read()
f.close()

and split the text file into pages:

In [ ]:
pattern = re.compile(r'[pP]age\s+[0-9]+')
pages = re.split(pattern, text)

How many pages are there in the text?

In [ ]:
len(pages)

Let's split each page into its words using `split()`. 

Also, we can create a dictionary `words_by_page`, so that we can look up the words in a page by page number. 


In [ ]:
words_by_page = {}
page_number = 1

all_words = []

for page in pages:
    split_page = page.split()
    # Adds the list of words to our dictionary
    words_by_page[page_number] = split_page
    page_number += 1
    all_words.extend(split_page)

What is the total number of words in the text?

In [ ]:
len(all_words)

To get a list of all the words on a certain page, index the `words_by_page` dictionary with square brackets [] 

In [ ]:
# Words on page 8
words_by_page[8]

Now we can compare the frequencies of words within pages and between pages.

`Counter` is a function that, as the name suggests, counts the number of appearances of each word in the list.

In [ ]:
from collections import Counter

#Yields the 10 most common words in page 8
Counter(words_by_page[8]).most_common(10)

As an exercise, write code below that yields the 8 most common words on page 30 

In [ ]:
# Your code here


You can look up how many times a certain word appears in a page, using `Counter`. Say we wanted to know how many times "þis" appears on page 34

In [ ]:
Counter(words_by_page[34])["þis"]

Let's create a function that gives us a DataFrame containing the frequencies per page of a given word. This way, we can compare frequencies directly between pages directly.

In [ ]:
def word_counts(word):
    """Returns a DataFrame containing the frequency counts 
    per page of the given word"""
    
    word_df = pd.DataFrame()
    word_df["Page"] = np.arange(1, len(pages)+1)
    
    word_counts = []
    
    # Goes page by page, calculating the counts of the word
    for page_num in word_df["Page"]:
        count = Counter(words_by_page[page_num])[word]
        word_counts.append(count)
    
    word_df["Frequency"] = np.array(word_counts)
    return word_df
    

Let's use `word_counts` to see how many times "þis" appears on each page.

In [ ]:
word_counts("þis")